In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools


The *has_infrequent_subset* function plays a crucial role in optimizing the Apriori algorithm **by reducing the number of candidate itemsets that need to be considered**, therefore **lowering the number of database scan operations.**


*   The function efficiently prunes candidate itemsets **by checking if any of their subsets are infrequent**. If any subset of a candidate itemset is infrequent, then the candidate itemset itself **cannot be frequent according to the Apriori property**.

*   By avoiding the generation of candidate itemsets containing infrequent subsets, **the search space is significantly reduced**. This leads to fewer candidate itemsets to be considered for support counting, resulting in overall efficiency gains.



*   Pruning candidate itemsets early in the process helps **conserve memory and processing resources**. It reduces the need to store and process large sets of candidate itemsets **that are unlikely to be frequent**.






In [10]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('bread-basket.csv')

# Split the "Item" column by commas and convert it to a list of lists
product_column = df["Item"].str.split(',').tolist()

# Find the maximum length of a transaction in the product_column
max_length = max(len(transaction) for transaction in product_column)

# Pad each transaction to have the same length using a placeholder value (e.g., 'None')
padded_product_column = [transaction + [''] * (max_length - len(transaction)) for transaction in product_column]

# Convert the padded_product_column to a NumPy array
product_array = np.array(padded_product_column)

# Example usage
print(product_array)


[['Bread']
 ['Scandinavian']
 ['Scandinavian']
 ...
 ['Coffee']
 ['Pastry']
 ['Smoothies']]


In [4]:
def generate_candidate_itemsets(itemset, k):
    """
    Generate candidate itemsets of size k from the given itemset.

    1- Initialize an empty list candidate_itemsets to store the generated candidate itemsets.
    2- Get the number of transactions (num_itemsets) in the itemset.
    3- Iterate through each pair of transactions using two nested loops (i and j).
    4- For each pair of transactions, check if their first k-1 items are the same. This is done using the condition itemset[i][:-1] == itemset[j][:-1].
    5- If the first k-1 items are the same, join the sets of items from both transactions and sort them. This is done using the expression sorted(set(itemset[i]) | set(itemset[j])).
    6- Check if the length of the resulting candidate itemset is equal to k.
    7- If the length is equal to k, check if any subset of the candidate itemset is infrequent. This is done using the has_infrequent_subset function.
    8- If none of the subsets are infrequent, append the candidate itemset to the candidate_itemsets list.
    9- Repeat this process for all pairs of transactions.
   10- Return the list of candidate itemsets.

   yginiri les combinaisons, yvirifyi ldakhel ida kayen item machi frequent mayajoutihch to candidate itemset 
   
    """
    candidate_itemsets = []
    num_itemsets = len(itemset)

    for i in range(num_itemsets):
        for j in range(i + 1, num_itemsets):
            if itemset[i][:-1] == itemset[j][:-1]:
                candidate = sorted(set(itemset[i]) | set(itemset[j]))
                if len(candidate) == k and '' not in candidate:
                    if not has_infrequent_subset(candidate, itemset):
                        candidate_itemsets.append(candidate)

    return candidate_itemsets

    """
    Checks whether all subsets of an itemset are frequent.

    1- Use itertools.combinations to generate all possible subsets of the itemset with length len(itemset) - 1.
    2- For each subset generated:
      2.1- Convert it to a list and check if it exists in the list of prev_itemsets.
      2.2- If any subset is not found in prev_itemsets, return True immediately, indicating that the itemset may contain infrequent subsets.
    3- If all subsets are found in prev_itemsets, return False, indicating that the itemset contains only frequent subsets.

    tvirfyi ida kayen item machi frequant
    
    """

def has_infrequent_subset(itemset, prev_itemsets):
    subsets = itertools.combinations(itemset, len(itemset) - 1)
    for subset in subsets:
        if list(subset) not in prev_itemsets:
            return True
    return False

In [5]:

    """
    Prune candidate itemsets that do not meet the minimum support threshold.

    1- Initialize an empty list freq_itemsets to store frequent itemsets.
    2- Initialize an empty dictionary item_counts to store the count of each candidate itemset.
    3- Iterate through each transaction in the itemset.
    4- For each transaction, iterate through each candidate itemset in candidate_itemsets.
    5- If the candidate itemset is a subset of the transaction, update its count in the item_counts dictionary.
    7- Iterate through each candidate itemset in candidate_itemsets.
    8- Calculate the support for each candidate itemset by dividing its count by the total number of transactions.
    9- If the support is greater than or equal to the min_support, add the candidate itemset to the freq_itemsets list.
   10- Return the list of frequent itemsets.

    """
    def prune_itemsets(itemset, candidate_itemsets, min_support, min_confidence):
      freq_itemsets = []
      item_counts = {}

      # Count the occurrences of each candidate itemset
      for transaction in itemset:
          for candidate in candidate_itemsets:
              if set(candidate).issubset(set(transaction)):
                  item_counts[str(candidate)] = item_counts.get(str(candidate), 0) + 1

      num_transactions = len(itemset)
      for candidate in candidate_itemsets:
          support = item_counts.get(str(candidate), 0) / num_transactions

          # Check if the support meets the minimum support threshold
          if support >= min_support:
              freq_itemsets.append(candidate) 

              # Calculate confidence for association rules
              for i in range(1, len(candidate)):
                  antecedent = candidate[:i]
                  consequent = candidate[i:]

                  antecedent_support = item_counts.get(str(antecedent), 0) / num_transactions
                  confidence = support / antecedent_support

                  # Check if confidence meets the minimum confidence threshold
                  if confidence >= min_confidence:
                      freq_itemsets.append([antecedent, consequent, confidence])#change

      return freq_itemsets


In [6]:
def apriori(itemset, min_support, min_conf):
    """
    Implement Apriori algorithm to find frequent itemsets.

    1- Initialize an empty list freq_itemsets to store frequent itemsets.
    2- Initialize k to 1.
    3- Extract unique items from itemset.
    4- Convert each unique item into a singleton itemset.
    5- Find frequent itemsets of size 1 by pruning using the prune_itemsets function.
    6- Append the frequent itemsets of size 1 to freq_itemsets.
    7- Iterate until no more frequent itemsets can be found:
    8- Generate candidate itemsets of size k + 1 using the generate_candidate_itemsets function.
    9- Find frequent itemsets of size k + 1 by pruning using the prune_itemsets function.
   10- Increment k.
   11- Append the frequent itemsets of size k + 1 to freq_itemsets.
   12- Return the list of frequent itemsets, excluding the last empty list.

    """
    freq_itemsets = []
    k = 1

    unique_items = set([item for sublist in itemset for item in sublist])
    unique_itemsets = [[item] for item in unique_items]
    freq_itemsets.append(prune_itemsets(itemset, unique_itemsets, min_support, min_conf))

    while freq_itemsets[-1] != []:
        candidate_itemsets = generate_candidate_itemsets(freq_itemsets[-1], k + 1)
        freq_itemsets.append(prune_itemsets(itemset, candidate_itemsets, min_support, min_conf))
        k += 1
    #boucle w7doukhra tvirifyi lconfidence
    return freq_itemsets[:-1]

In [11]:
if __name__ == "__main__":
    # Example dataset

    min_support = 0.01
    min_confidence = 0.01
    freq_itemsets = apriori(product_array, min_support, min_confidence)
    print("Frequent Itemsets:", freq_itemsets)

Frequent Itemsets: [[['Brownie'], ['Bread'], ['Scone'], ['Cake'], ['Soup'], ['Juice'], ['Muffin'], ['Sandwich'], ['Farm House'], ['Pastry'], ['Toast'], ['Coffee'], ['Hot chocolate'], ['Scandinavian'], ['Cookies'], ['Tea'], ['Alfajores'], ['Medialuna']]]
